In [1]:
!python3 -m pip install --upgrade pip

In [2]:
# !pip install openai==1.2.3
!pip install openai==1.3.4
!pip install python-dotenv tiktoken
!pip install pdfplumber
!pip install arxiv==2.1.0

In [3]:
from time import time

class Timer:
    def __init__(self, logger=None, format_str="{:.3f}[s]", prefix=None, suffix=None, sep=" "):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [4]:
import re
# 取得したいURL
url = "https://arxiv.org/abs/2403.10351v1"

identifier = re.search(r'/([^/]+)$', url).group(1)
identifier

'2403.10351v1'

In [5]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [6]:
import arxiv
import re

def get_arxiv_info_by_url(url):
    # URLからタイトルと概要を取得する関数
    # URLから論文のIDを抽出する
    match = re.search(r'arxiv\.org/abs/([0-9\.v]+)', url)
    if not match:
        return "URLから論文のIDを抽出できませんでした。"
    
    arxiv_id = match.group(1)
    
    # arXivから論文情報を検索するためのクライアントと検索条件を設定
    client = arxiv.Client()
    search = arxiv.Search(id_list=[arxiv_id])
    
    # 検索結果を取得
    result = next(client.results(search), None)
    
    if result:
        # 論文のタイトルと概要を返す
        return {
            "title": result.title,
            "summary": result.summary
        }
    else:
        return {"title": "指定されたIDの論文が見つかりませんでした。"}

In [7]:
info = get_arxiv_info_by_url(url)
print(info)

{'title': 'TriSum: Learning Summarization Ability from Large Language Models with Structured Rationale', 'summary': "The advent of large language models (LLMs) has significantly advanced natural\nlanguage processing tasks like text summarization. However, their large size\nand computational demands, coupled with privacy concerns in data transmission,\nlimit their use in resource-constrained and privacy-centric settings. To\novercome this, we introduce TriSum, a framework for distilling LLMs' text\nsummarization abilities into a compact, local model. Initially, LLMs extract a\nset of aspect-triple rationales and summaries, which are refined using a\ndual-scoring method for quality. Next, a smaller local model is trained with\nthese tasks, employing a curriculum learning strategy that evolves from simple\nto complex tasks. Our method enhances local model performance on various\nbenchmarks (CNN/DailyMail, XSum, and ClinicalTrial), outperforming baselines by\n4.5%, 8.5%, and 7.4%, respecti

In [8]:
import requests

def download_pdf(link, save_path):
    response = requests.get(link)
    with open(save_path, 'wb') as file:
        file.write(response.content)

In [9]:
import os

files_to_remove = ['sample.pdf']

for file in files_to_remove:
    if os.path.exists(file):
        os.remove(file)
        print(f"{file} を削除しました。")
    else:
        print(f"{file} は存在しません。")

sample.pdf を削除しました。


In [10]:
url = "https://arxiv.org/pdf/" + identifier

with Timer(prefix="download_pdf"):
    # download_pdf(url, nougat_path)
    download_pdf(url, "sample.pdf")

download_pdf 1.823[s]


In [11]:
pdf_path = "./sample.pdf"

In [12]:
import openai
import pdfplumber
from openai import OpenAI
import tiktoken
from dotenv import load_dotenv
import os
import json

load_dotenv()

True

In [13]:
file_path = 'sample.pdf'  # PDFファイルのパス

In [14]:
# PDFファイルを読み込み、テキストに変換する関数
def pdf_to_text(file_path):
    with pdfplumber.open(file_path) as pdf:
        return ''.join(page.extract_text() for page in pdf.pages)


In [15]:
with Timer(prefix="pdf_to_text"):
    paper_text = pdf_to_text(file_path)

pdf_to_text 3.795[s]


In [16]:
print(paper_text)

TriSum: Learning Summarization Ability from Large Language Models
with Structured Rationale
PengchengJiang1,CaoXiao2,ZifengWang1,ParminderBhatia2,
JimengSun1,andJiaweiHan1
1UniversityofIllinoisatUrbana-Champaign
2GEHealthCare
1{pj20, zifengw2, jimeng, hanj}@illinois.edu
2danicaxiao@gmail.com
Abstract Step 1: Step 2: Step 3:
LLM Rationale Golden Rationale Local Training
Probing Selection
Theadventoflargelanguagemodels(LLMs) Document Prompt Document
has significantly advanced natural language Scoring
processingtasksliketextsummarization. How-
ever, their large size and computational de- ? LLM MSm oda ell l
mands, coupled with privacy concerns in
datatransmission, limittheiruseinresource- Rationale Golden
Ground-Truth Candidates Rationale
constrained and privacy-centric settings. To Summary
overcomethis,weintroduceTriSum,aframe-
Figure1: Aconceptualdemonstrationofourthree-step
workfordistillingLLMs’textsummarization
framework TriSum that endows local small models
abilitiesintoacompact,loc

In [17]:
# MODEL_NAME = "gpt-3.5-turbo-0125"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
MODEL_NAME = "gpt-4-0125-preview"
TEMPERATURE = 0.7
# OpenAIクライアントの初期化
client = OpenAI()

In [18]:
def ground_truth_summary(model_name, doc):
    # 文書を要約するプロセスにおける、ゼロショット設定での要約生成のタスク
    prompt = [{'role': 'system', 'content': "Given a document, summarize the document in"}]
    prompt.append({"role" : "system", "content" : 'one sentence: Given a document, summarize the document in'})
    prompt.append({"role" : "system", "content" : f'three sentence: Document: {doc}'})
    prompt.append({"role": "user", "content": f"Summary:"})
    """
    文書を与えられた場合、その文書を1文で要約します。
    文書を与えられた場合、その文書を3文で要約します。
    文書: {doc}
    要約:
    """
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    response = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        temperature=TEMPERATURE,
    )
    summary_str = response.choices[0].message.content
    
    return summary_str

In [19]:
# LLM理由付け探索 (LLM Rationale Probing)
with Timer(prefix="LLM Rationale Probing"):
    gt_summary = ground_truth_summary(MODEL_NAME, paper_text)

LLM Rationale Probing 11.475[s]


In [20]:
print(gt_summary)

TriSum is a framework designed to distill text summarization capabilities from large language models (LLMs) into smaller, local models, enhancing performance and interpretability across various datasets. Through a three-step process involving rationale probing, golden rationale selection, and curriculum learning, TriSum systematically improves summarization quality while offering insights into the rationale behind model decisions. Extensive testing across datasets like CNN/DailyMail, XSum, and ClinicalTrial demonstrates TriSum's superiority over state-of-the-art models and its adaptability to different summarization tasks. Additionally, TriSum's methodology of utilizing aspect-triple rationales not only aids in generating more accurate and factually consistent summaries but also highlights the potential for scaling this approach to other summarization models and tasks.


In [21]:
def rationale_probing(model_name, doc, gt_summary):
    # TriSum: LLM理由付け探索 (LLM Rationale Probing)
    # 与えられた文書に基づいて重要な側面とそれらに関連する三重項（エンティティとその関係）を抽出し、これらを使用して要約を生成
    prompt = [{'role': 'system', 'content': "Given a document and its ground-truth summary, do"}]
    prompt.append({"role" : "system", "content" : 'the following tasks:'})
    prompt.append({"role" : "system", "content" : '(1) According to the Ground-truth Summary, extract essential aspects of the Document.'})
    prompt.append({"role" : "system", "content" : '(2) For each essential aspect, retrieve detailed triples in the format [ENTITY1 | RELATION | ENTITY2] used to compose the ground-truth summary.'})
    prompt.append({"role" : "system", "content" : '(3) With the retrieved triples, compose a summary.The essential aspects, triples, and composed summary should be in the same response, separated by a new line.'})
    prompt.append({"role" : "system", "content" : 'All triples [ENTITY1 | RELATION | ENTITY2] should be in length 3 (separated by "|").'})
    prompt.append({"role" : "system", "content" : 'Example:\n\n================Example=================\nPrompt:\n[Document]: [document]\n[Ground-truth Summary]: [ground-truth summary]\nUpdate:\nEssential Aspects:\n[aspects]\nTriples:\n- [ENTITY1_1 | RELATION_1 | ENTITY1_2]\n- [ENTITY2_1 | RELATION_2 | ENTITY2_2]\n- [ENTITY3_1 | RELATION_3 | ENTITY3_2]\n- ...\nGenerated Summary:\n[summary]\n========================================'})
    
    prompt.append({"role": "system", "content": "Please format the output in Markdown."})
    prompt.append({"role": "system", "content": "Results must be in Japanese."})
    
    prompt.append({"role": "user", "content": f"Prompt:"})
    prompt.append({"role": "user", "content": f"[Document]: {doc}"})
    prompt.append({"role": "user", "content": f"[Ground-truth Summary]: {gt_summary}"})
    prompt.append({"role": "user", "content": f"Update:"})
    """
    システム

    文書とその正確な要約が与えられた場合、以下のタスクを実行してください。
    (1) 正確な要約に基づき、文書の重要な側面を抽出します。
    (2) それぞれの重要な側面について、正確な要約を構成するために使用された[ENTITY1 | RELATION | ENTITY2]の形式で詳細な三つ組を取得します。
    (3) 取得した三つ組を用いて、要約を作成します。重要な側面、三つ組、および作成した要約は、新しい行で区切られ、同じ返答の中に含めてください。
    全ての三つ組[ENTITY1 | RELATION | ENTITY2]は、3つの長さである必要があります（"|"で区切られる）。
    例:
    ================例=================
    プロンプト:
    [文書]: [document]
    [正確な要約]: [ground-truth summary]
    更新:
    重要な側面:
    [aspects]
    三つ組:

    [ENTITY1_1 | RELATION_1 | ENTITY1_2]
    [ENTITY2_1 | RELATION_2 | ENTITY2_2]
    [ENTITY3_1 | RELATION_3 | ENTITY3_2]
    ...
    生成された要約:
    [summary]
    ========================================
    プロンプト:
    [文書]: {doc}
    [正確な要約]: {gt_summary}
    更新:
    """
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    response = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        temperature=TEMPERATURE,
    )
    summary_method_name_str = response.choices[0].message.content
    
    return summary_method_name_str

In [22]:
# LLM理由付け探索 (LLM Rationale Probing)
with Timer(prefix="LLM Rationale Probing"):
    # summary_method_name = rationale_probing(MODEL_NAME, paper_text, info["summary"])
    summary_method_name = rationale_probing(MODEL_NAME, paper_text, gt_summary)

LLM Rationale Probing 41.289[s]


In [23]:
summary_method_name

'Essential Aspects:\n- TriSum framework\n- Distillation process from LLMs to local models\n- Performance and interpretability enhancement\n- Three-step process: rationale probing, golden rationale selection, curriculum learning\n- Testing across CNN/DailyMail, XSum, and ClinicalTrial datasets\n- Superiority over state-of-the-art models\n- Utilization of aspect-triple rationales for accuracy and factual consistency\n\nTriples:\n- [TriSum | is designed to | distill text summarization capabilities]\n- [TriSum | enhances | performance and interpretability]\n- [Process | involves | rationale probing, golden rationale selection, curriculum learning]\n- [TriSum | tested across | CNN/DailyMail, XSum, and ClinicalTrial]\n- [TriSum | demonstrates | superiority over state-of-the-art models]\n- [TriSum | utilizes | aspect-triple rationales for accuracy and factual consistency]\n\nGenerated Summary:\nTriSumは、大規模な言語モデル（LLM）から小規模なローカルモデルへのテキスト要約能力の蒸留を目的としたフレームワークであり、CNN/DailyMail、XSum、ClinicalTrialなど

In [24]:
print(summary_method_name)

Essential Aspects:
- TriSum framework
- Distillation process from LLMs to local models
- Performance and interpretability enhancement
- Three-step process: rationale probing, golden rationale selection, curriculum learning
- Testing across CNN/DailyMail, XSum, and ClinicalTrial datasets
- Superiority over state-of-the-art models
- Utilization of aspect-triple rationales for accuracy and factual consistency

Triples:
- [TriSum | is designed to | distill text summarization capabilities]
- [TriSum | enhances | performance and interpretability]
- [Process | involves | rationale probing, golden rationale selection, curriculum learning]
- [TriSum | tested across | CNN/DailyMail, XSum, and ClinicalTrial]
- [TriSum | demonstrates | superiority over state-of-the-art models]
- [TriSum | utilizes | aspect-triple rationales for accuracy and factual consistency]

Generated Summary:
TriSumは、大規模な言語モデル（LLM）から小規模なローカルモデルへのテキスト要約能力の蒸留を目的としたフレームワークであり、CNN/DailyMail、XSum、ClinicalTrialなどのさまざまなデータセットでのパフォーマ

In [25]:
def explain_method_algorithm(model_name, text, method_name):
    # アルゴリズムの説明
    prompt = [{'role': 'system', 'content': "Please explain the algorithm of the method name from the following text in detail, using both sentences and formulas. Carefully describe the mechanism in order, ensuring that it can be understood and implemented. Design the process flow in a way that allows for the algorithm to be implemented without any omissions or excess."}]
    prompt.append({"role" : "system", "content" : "Describe the algorithm in detail, explaining what it aims to achieve, how it processes to accomplish this, and how exactly these processes are carried out, regardless of the length of the explanation. Just ensure it is accurate."})
    prompt.append({"role" : "system", "content" : "Outputs should be generated in step by step."})
    prompt.append({"role" : "system", "content" : "Please format the output in Markdown."})
    prompt.append({"role": "system", "content": "Results must be in Japanese."})
    prompt.append({"role": "system", "content": 'Please generate a JSON from the following input text. Use "method" as the schema, and for the key, use "the detailed explanation of the processing of the method_name algorithm in simple language". Generate it in the format of {"method": "the result of a detailed explanation of the method_names algorithm described in simple language"}.'})
    
    prompt.append({"role": "user", "content": 'Generate a JSON from the following input text. Use "method" as the schema, and use the judgment result as the key, to create it in the format {"method": the result of grouping the search_query based on relevance into a list format that can be used in Python}.'})
    
    prompt.append({"role": "user", "content": f"Input text: {text}"})
    prompt.append({"role": "user", "content": f"method name: {method_name}"})
    
    """
    システム
    あなたは以下の text から method name のアルゴリズムを順番に過不足なく文章と数式で丁寧に順番に仕組みが理解でき、実装をするための処理の流れを設計できるように説明してください。
    アルゴリズムの説明は、何を実現するために、どのように処理を実行し、その処理はどのように実行されるのかをどれだけ長くなってもよいのでとにかく正確に説明してください。
    出力は Markdown 形式にしてください。
    
    結果は日本語でなければならない。
    以下の入力テキストからJSONを生成してください。スキーマには "method"、キーには"text から method_name のアルゴリズムを平易な文章で処理内容を詳細に説明した内容"を使ってください。"method": "method_name のアルゴリズムを平易な文章で処理内容を詳細に説明した内容した結果}'}の形式で生成してください。
    user
    
    入力されたテキスト: {text}
    method name: {method_name}
    """
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    method = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    method_str = method.choices[0].message.content
    
    # JSON形式の文字列を辞書に変換
    method_algorithm = json.loads(method_str)
    
    # 出力と新しいメッセージをステートに反映
    return {
        "method": method_algorithm["method"],
    }


In [26]:
with Timer(prefix="explain_method_algorithm"):
    # method_algorithm = explain_method_algorithm(MODEL_NAME, paper_text, str(summary_method_name["method_name"]))
    method_algorithm = explain_method_algorithm(MODEL_NAME, paper_text, summary_method_name)

explain_method_algorithm 26.765[s]


In [27]:
print(method_algorithm["method"])

TriSumは、大規模な言語モデル（LLM）から小規模なローカルモデルへのテキスト要約能力の蒸留を目的としたフレームワークであり、CNN/DailyMail、XSum、ClinicalTrialなどのさまざまなデータセットでのパフォーマンスと解釈可能性を向上させます。理由の探求、黄金の理由の選択、カリキュラム学習という3つのステップを通じて、TriSumは要約品質を体系的に改善し、モデルの決定背後にある理由に洞察を提供します。また、アスペクト-トリプルの理由を利用することで、より正確で事実に基づいた要約を生成するだけでなく、このアプローチを他の要約モデルやタスクにスケーリングする可能性を示しています。


In [28]:
def generate_pseudocode_for_method(model_name, algorithm):
    # アルゴリズムの説明
    prompt = [{'role': 'system', 'content': "Based on the description of the following algorithm, please create a comprehensive pseudo-implementation code in Python without omitting any details."}]
    prompt.append({"role" : "system", "content" : "Outputs should be generated in step by step."})
    prompt.append({"role": "system", "content": "Please format the output in Markdown."})
    prompt.append({"role": "system", "content": "Comment must be in Japanese."})
    prompt.append({"role": "system", "content": 'Please generate a JSON from the following input text. Use "code" as the schema, and for the key, use "the result of generating code that executes the algorithm of algorithm in Python". Generate it in the format of {"code": "the result of reproducing the algorithm algorithm in Python code"}.'})
        
    prompt.append({"role": "user", "content": f"algorithm: {algorithm}"})
    
    """
    システム
    あなたは以下の algorithm の説明を基にpythonの疑似実装コードを過不足なく作成してください。
    出力は Markdown 形式にしてください。
    
    コメントは日本語でなければならない。
    以下の入力テキストからJSONを生成してください。スキーマには "code"、キーには"algorithm のアルゴリズムをpythonのコードで動くようにコードを生成した結果"を使ってください。"code": "algorithm のアルゴリズムをpythonのコードで再現した結果"}'}の形式で生成してください。
    user
    
    algorithm: {algorithm}
    """
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    code_res = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    code_str = code_res.choices[0].message.content
    print(code_str)
    
    # JSON形式の文字列を辞書に変換
    code = json.loads(code_str)
    
    # 出力と新しいメッセージをステートに反映
    return {
        "code": code["code"],
    }


In [29]:
with Timer(prefix="generate_pseudocode_for_method"):
    code_method = generate_pseudocode_for_method(MODEL_NAME, str(method_algorithm["method"]))


{
  "code": "# TriSumアルゴリズムの実装\n\n# 必要なライブラリのインポート\nimport numpy as np\n\n# TriSumフレームワークの概要\n# TriSumは、大規模な言語モデルから小規模なローカルモデルへのテキスト要約能力の蒸留を目的としたフレームワークです。\n# これは、CNN/DailyMail、XSum、ClinicalTrialなどのさまざまなデータセットでのパフォーマンスと解釈可能性を向上させます。\n\n# TriSumの3つの主要なステップ\n\n# 1. 理由の探求\n# コメント: 大規模な言語モデルを使って、テキストから重要な理由を抽出します。\n\n# 2. 黄金の理由の選択\n# コメント: 抽出した理由の中から最も重要なものを選択します。\n\n# 3. カリキュラム学習\n# コメント: 選択した理由を使って、小規模なローカルモデルを段階的に訓練します。\n\n# コードの具体的な実装は、特定のモデルやデータセットに依存しますが、このフレームワークの概念は、\n# 要約品質を体系的に改善し、モデルの決定背後にある理由に洞察を提供するために使用できます。\n\n# アスペクト-トリプルの理由を利用することで、より正確で事実に基づいた要約を生成することが可能になります。\n# また、このアプローチは他の要約モデルやタスクにもスケーリングする可能性を示しています。\n\n# 注意: このコードは、TriSumフレームワークの概念と主要なステップを説明するものであり、\n# 実際の実装には特定のモデルやデータセット、およびそれらの処理に関する詳細なコードが必要になります。"
}
generate_pseudocode_for_method 36.577[s]


In [30]:
print(code_method["code"])

# TriSumアルゴリズムの実装

# 必要なライブラリのインポート
import numpy as np

# TriSumフレームワークの概要
# TriSumは、大規模な言語モデルから小規模なローカルモデルへのテキスト要約能力の蒸留を目的としたフレームワークです。
# これは、CNN/DailyMail、XSum、ClinicalTrialなどのさまざまなデータセットでのパフォーマンスと解釈可能性を向上させます。

# TriSumの3つの主要なステップ

# 1. 理由の探求
# コメント: 大規模な言語モデルを使って、テキストから重要な理由を抽出します。

# 2. 黄金の理由の選択
# コメント: 抽出した理由の中から最も重要なものを選択します。

# 3. カリキュラム学習
# コメント: 選択した理由を使って、小規模なローカルモデルを段階的に訓練します。

# コードの具体的な実装は、特定のモデルやデータセットに依存しますが、このフレームワークの概念は、
# 要約品質を体系的に改善し、モデルの決定背後にある理由に洞察を提供するために使用できます。

# アスペクト-トリプルの理由を利用することで、より正確で事実に基づいた要約を生成することが可能になります。
# また、このアプローチは他の要約モデルやタスクにもスケーリングする可能性を示しています。

# 注意: このコードは、TriSumフレームワークの概念と主要なステップを説明するものであり、
# 実際の実装には特定のモデルやデータセット、およびそれらの処理に関する詳細なコードが必要になります。
